In [1]:
import sys
sys.path.append("E:/Project/ZLTProgram/CelebA")
from Utils.dataload import CELEBA,drawCelebAImages
from torchvision import transforms
import torch
from Model.function import sample_z
import numpy as np
# 数据集存储目录
root='E:/Project/ModelAndDataset/data'
attr_name='Smiling'
batch_size=64
# 模型参数存放目录
save_model_dir='E:/Project/ModelAndDataset/model/CelebA/cVAE_GAN'
testDataset = CELEBA(root=root, train=False,train_ratio=0.7 ,transform=transforms.ToTensor(),label=attr_name)
testLoader = torch.utils.data.DataLoader(testDataset, batch_size=batch_size, shuffle=False)

31
Load Test dataset: 60780


In [2]:
from Model.CVAE_GAN_Advanced import CVAE
from os.path import join
#GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
nz=100
fsize=64
cvae = CVAE(nz=nz, imSize=64,self_attn=True,CBN=True,fSize=fsize,device=device)
cvae.to(device)
cvae.load_params(modelDir=join(save_model_dir,attr_name))

loading params...


In [3]:
pics,y=next(iter(testLoader))

mu, log_var, _= cvae.encode(pics)
z = cvae.re_param(mu, log_var)
# 生成独热编码矩阵
yRand = torch.eye(2)[torch.LongTensor(y.data.cpu().numpy())].type_as(z)
yOnes = torch.eye(2)[torch.LongTensor(np.ones(y.size()).astype(int))].type_as(z)
yZeros = torch.eye(2)[torch.LongTensor(np.zeros(y.size()).astype(int))].type_as(z)

gen_pics=cvae.decode(y=yRand.to(device),z=z.to(device))
gen_pics0=cvae.decode(y=yZeros.to(device),z=z.to(device))
gen_pics1=cvae.decode(y=yOnes.to(device),z=z.to(device))

recon_x,_,_,_=cvae(pics)

drawCelebAImages(pics.detach().cpu().numpy(),
                 y.cpu(),
                 attr_name,
                 "E:\Project\ZLTProgram\Images\gen_test/real_image.png",
                 )

drawCelebAImages(gen_pics.detach().cpu().numpy(),
                 y.cpu(),
                 attr_name,
                 "E:\Project\ZLTProgram\Images\gen_test/imgae_label.png",
                 )
drawCelebAImages(gen_pics.detach().cpu().numpy(),
                 y.cpu(),
                 attr_name,
                 "E:\Project\ZLTProgram\Images\gen_test/imgae_label.png",
                 )
drawCelebAImages(gen_pics0.detach().cpu().numpy(),
                 np.zeros(y.size()).astype(int),
                 attr_name,
                 "E:\Project\ZLTProgram\Images\gen_test/negative.png",
                 )
drawCelebAImages(gen_pics1.detach().cpu().numpy(),
                 np.ones(y.size()).astype(int),
                 attr_name,
                 "E:\Project\ZLTProgram\Images\gen_test/positive.png",
                 )
drawCelebAImages(recon_x.detach().cpu().numpy(),
                 y.cpu(),
                 attr_name,
                 "E:\Project\ZLTProgram\Images\gen_test/recon_img.png",
                 )
drawCelebAImages((gen_pics1-gen_pics0).detach().cpu().numpy(),
                 y.cpu(),
                 attr_name,
                 "E:\Project\ZLTProgram\Images\gen_test/difference.png",
                 )






e:\Project\ZLTProgram\CelebA\Model\CVAE_GAN_Advanced.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y = F.softmax(self.encY(x.detach()))


Image E:\Project\ZLTProgram\Images\gen_test/imgae_label.png exists.
Image E:\Project\ZLTProgram\Images\gen_test/imgae_label.png exists.
Image E:\Project\ZLTProgram\Images\gen_test/negative.png exists.
Image E:\Project\ZLTProgram\Images\gen_test/positive.png exists.
Image E:\Project\ZLTProgram\Images\gen_test/recon_img.png exists.


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i